# AI Workshop - Lab 2-1: Decision Trees and Neural Networks

In the first lab, we used a decision tree classifier to evaluate the quality of different imputation methods, but we didn't really discuss what decision trees are or how they work. In this lab, we will delve into the details of decision trees and explore how they can be used to make predictions with our energy and weather datasets.

In [ ]:
# Install graphviz, which we will use later in the lab
!pip install -Uq graphviz

### Decision Trees

Decision trees are popular supervised learning methods used for classification and regression. The tree represents a series of simple decision rules that predict the target when the feature vector is passed through them. Decision trees are easy to understand, can be visualized nicely, require very little data preparation (e.g., we don't need to scale features), and the trained model can be explained easily to others post priori (as opposed to other *black box* methods that are difficult to communicate).

###### Example
Suppose you wanted to design a simple decision tree for whether (or not) you buy a used car. You might develop something like the following:

<img src="https://github.com/lyeskhalil/mlbootcamp/blob/master/img/decision-tree.gif?raw=1" width="500"/>

**YOUR TURN:** Let's say you're browsing Kijiji and come across a used car that: has been road tested, has high mileage, and is a recent year/model.
* According to your decision tree model, should you buy this car or not? ____________________________
* Will you buy any cars that haven't been road tested (if you follow your model)? ___________________________________

Obviously this tree may not be ideal, depending on the situation. For example, you could have a road tested car of a recent year with 2,000,000 km's on it and the model is telling you to buy! (But, you probably shouldn't)

### Energy and Weather Datasets

Just as in the first lab, we are going to use the energy and weather datasets to explore decision trees. To save time, we have prepared a cleaned version that we will use this afternoon. Let's load the data and take a look at the first few rows.

In [ ]:
import pandas as pd

df = pd.read_csv('data/cleaned_energy_data.csv')
df

As we have seen already, we can use Pandas to conveniently summarize key aspects of the data. For example, we can use the `describe` method to get a quick statistical summary of the data:


In [ ]:
df.describe()

##### Model Development

In the previous lab, we looked at predicting weather conditions based on energy demand. We found that there are too many types of condition for a simple model to predict accurately. This time, we'll look specifically at the presence or absence of clouds using the `clouds_all` column (which records the percentage cloud cover at the time of the observation).

Let's import sklearn's decision tree classifer and split the data (using techniques we covered in the first lab).

In [ ]:
from sklearn.model_selection import train_test_split

target_data = df['clouds_all'] != 0
feature_data = df[['generation biomass', 'generation fossil brown coal/lignite',
                   'generation fossil gas',
                   'generation fossil hard coal', 'generation fossil oil',
                   'generation hydro pumped storage consumption',
                   'generation hydro run-of-river and poundage',
                   'generation hydro water reservoir',
                   'generation nuclear', 'generation other', 'generation other renewable',
                   'generation solar', 'generation waste',
                   'generation wind onshore']]

X_train, X_test, y_train, y_test = train_test_split(feature_data, target_data, test_size=0.3, random_state=0)

**YOUR TURN:**
* How many samples are in the training set? _______________________
* How many samples are in the test set? _______________________
* What percentage of the samples have clouds in the training set? _______________________

In [ ]:
## Your code here
print(f'Training samples: {len(X_train)}')
print(f'Test samples: {len(X_test)}')
print(f'Percentage of samples with clouds in the training set: {y_train.mean()}')

##### Dealing with Missing Data: Imputation

Before we can fit our decision tree to our training data, we can conduct imputation to replace missing values in our dataset. Previously we did this manually so that we could follow the process, but now we'll use the handy `SimpleImputer` class from sklearn. This class allows us to replace missing values with a specified strategy (e.g., mean, median, most frequent).

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

imp.fit(X_train)
X_train = imp.transform(X_train) # replace missing data using our imputer

So we've got our data prepared, let's fit a decision tree to our training data.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth=10) # We'll set a max depth to prevent overfitting

clf.fit(X_train, y_train)

accuracy = accuracy_score(y_train, clf.predict(X_train))
print ("Accuracy: ", accuracy * 100, "%")

In the above cell, we defined a Decision Tree classifier and fit it to our training set. When we then used it to predict training set values, the resulting accuracy was ~63%

**YOUR TURN:**
* Since we are both training and predicting on our training set, why didn't the decision tree achieve 100% accuracy?
* What is the performance of this model on the test set?


In [ ]:
## Your code here
##
##
##

##### Feature Importances

One thing we can do is take a look at the relative feature importances of the trained decision tree classifier. This will give us an idea of what the model thinks is more/less important for properly predicting the target.

Let's look at the feature importances for a model on the training data:

In [ ]:
for feature, importance in zip(feature_data.columns, clf.feature_importances_):
    print(f'{feature:<43} {importance:0.2f}')

As we can see, the model places a higher importance on onshore nuclear generation than other features, although the importance is broadly spread across all features.

#### Visualizing the Tree

One useful thing we can do is actually visualize our decision tree model! We can use the [graphViz](https://www.graphviz.org/) library to accomplish this:

In [ ]:
from sklearn.tree import export_graphviz
import graphviz # Package containing visualization tools

export_graphviz(clf, out_file="mytree.dot", feature_names=feature_data.columns) # export the tree to .dot file
with open("mytree.dot") as f: # read the file back in
    dot_graph = f.read()
graphviz.Source(dot_graph) # display the tree

**YOUR TURN:** Explore the decision tree and answer the following:
* What feature does the root node split on?
* What is the depth of the decision tree (i.e., the length of the longest path from root to leaf)?
* Do you think this decision tree is prone to overfitting? Why/why not?

To reduce the degree to which this tree is overfit to the training data, we can force the tree to be of some *maximum depth*. This ensures the tree won't be able to just keep generating new layers to properly classify every sample in the training stage (and, thus, presumably generalize better to the test set).

Let's try limiting the max depth to 2 and visualizing the resulting tree.

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth = 2)

clf.fit(X_train, y_train)

export_graphviz(clf, out_file="mytree.dot", feature_names=feature_data.columns)
with open("mytree.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

Much simpler!

There are many hyper-parameters that can be tuned to change how the model performs. Some common parameters that are modified include:
1. Max Tree Depth: How "tall" do you want your tree to be
2. Minimum Samples Per Leaf: This parameter defines the minimum number of training datapoints that fall into a given leaf node in order for that node to be created
3. Minimum Samples to Split: This parameter controls the minimum number of samples required to create a decision split

To decide the values of each of the parameters, we can use Grid Search combined with cross validation. In Grid Search, we first decide what potential values we want each hyperparameter will take. Then we find every possible combination of parameters and run cross validation on each combination to estimate the performance of that hyperparameter combination.

Luckily, `sklearn` has a nice implementation of Grid Search that runs this algorithm for us. Lets see a demo below:

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

clf = tree.DecisionTreeClassifier() # First we define our model without passing in parameters
hyperparameter_search = { # Then we decide the possible parameter combinations
    'max_depth': [2, 3, 4, 5],
    'min_samples_split': [2, 5, 8, 11],
    'min_samples_leaf': [2, 5, 8, 11]
} # Since we have 3 parameters with 2 possible values, grid search will test 3^3 combinations
evaluation_metric = make_scorer(accuracy_score, # GridSearchCV requires us to wrap our metric function in a "scorer"
                                greater_is_better = True)

grid_search_cv = GridSearchCV(estimator = clf,
                              param_grid = hyperparameter_search,
                              scoring = evaluation_metric,
                              n_jobs = -1, # Use all available cores
                              cv = 5) # Set up search algorithm
grid_search_cv.fit(X_train, y_train) # Run the search. NOTE: This may take a while

print("Best Parameters: ", grid_search_cv.best_params_) # Print the parameters
print(f'Best Accuracy: {grid_search_cv.best_score_ * 100:0.2f}%') # Print the accuracy of the best model

clf = grid_search_cv.best_estimator_ # Get the best model from the GridSearch
accuracy = accuracy_score(y_test, clf.predict(imp.transform(X_test)))
print(f'Accuracy on test set: {accuracy * 100:0.2f}%')

In the cell above, we tested our two values per hyperparameter and ran grid search to find the best combination from the space we defined. As you may have noticed, the number of combinations tested by Grid Search exponentially increases as you test more values and tune more hyperparameters. This means that performing a grid search is often a task that takes a long period of time and is often note used for more complex models like neural networks.

# Neural Networks with Keras

Let's move on to making predictions on our data using neural networks. We will use the Keras library to build a simple neural network and evaluate its performance on the test set.

This time, we will use our data for a *regression* task instead of a classification task. We will predict the amount of energy consumed based on the weather conditions (perhaps more in line with what you were expecting all along!) We will use a simple feedforward neural network with one hidden layer.

## Data Preparation

Let's prepare our X and y data for this new task. We will predict the total energy consumed based on the weather conditions.

In [ ]:
target_data = df['total load actual']

feature_data = df[['city_name', 'temp', 'temp_min', 'temp_max', 'pressure',
                   'humidity', 'wind_speed', 'wind_deg', 'rain_1h', 'rain_3h', 'snow_3h',
                   'clouds_all', 'weather_main']]

# One-hot encode city_name, weather_main
feature_data = pd.get_dummies(feature_data, columns=['city_name', 'weather_main'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature_data, target_data, test_size=0.3, random_state=0)

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

imp.fit(X_train)
X_train = imp.transform(X_train) # replace missing data using our imputer
X_test = imp.transform(X_test)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Let's check our data to confirm it's what we expect.

In [ ]:
pd.DataFrame(X_train, columns=feature_data.columns)

In [ ]:
y_train

Looks good - we have our X and y data prepared for the regression task. Now we can build our neural network model!

## Building a Neural Network with Keras

Keras is a high-level neural networks API, written in Python and capable of running TensorFlow code. We will use Keras to build a simple feedforward neural network with one hidden layer. We will use the `Sequential` model from Keras to build our network.

Let's start by importing the necessary modules from Keras, and recording some of the data shapes (specifically, the number of features in our input data).

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Input

input_shape = X_train.shape[1]
output_shape = 1 # We are predicting a single value

Now we can build our model. We will use the `Sequential` model from Keras to build a simple feedforward neural network with one hidden layer. We will use the `Dense` class to define the layers in our network, which are fully connected layers.

Let's start by building a simple model with 1 hidden layer and 10 neurons. We need to make sure that the input shape of the first layer matches the number of features in our input data.

In [ ]:
model = Sequential(
    [
        Input(shape=(input_shape,)), # Defining the input layer
        Dense(10), # <--- Hidden layer with 10 neurons
        Activation('relu'), # Simple ReLU activation
        Dense(output_shape) # One output neuron
    ]
)

model.compile(optimizer='sgd', loss='mean_absolute_error') # Compile the model
model.summary()

We have built a simple neural network with 1 hidden layer and 10 neurons. The model has 301 parameters (weights) that need to be learned.

We defined our network to be trained using Stochastic Gradient Descent, the most basic optimization algorithm. We also defined the loss function to be the mean absolute error, which is commonly used for regression tasks.

Now we are ready to train our model! We will use the `fit` method to train the model on our training data.

In [ ]:
model.fit(
    X_train, y_train, # Training data
    epochs=5, # Number of epochs to train for
    batch_size=128, # Number of samples per gradient update
    validation_data=(X_test, y_test) # Validation data
)

As a final check, let's calculate both the Mean Absolute Error and the Mean Absolute Percentage Error on the testing set. As it sounds, the MAPE gives us the average percentage that the predictions differ from the true value, which is helpful for understanding the level of accuracy.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

y_pred = model.predict(X_test)
print(f'MAE:  {mean_absolute_error(y_test, y_pred):.02f}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)*100:.02f}%')